# Data Exploration Notebook

This notebook explores the retail sales data produced by the ETL pipeline.

> **Before running this notebook**, make sure you have run the pipeline first:
> ```bash
> python pipeline.py
> ```
> That will create the `database/retail.db` file and the processed CSVs this notebook reads from.

### What we'll cover
1. Load data from the SQLite database
2. Inspect the raw shape and types of each table
3. Summary statistics
4. Revenue and profit by category
5. Daily sales trend
6. Store performance comparison
7. Top products by revenue

In [ ]:
import sys
import sqlite3
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Make imports work whether the notebook is opened from the repo root
# or from inside the notebooks/ subdirectory.
ROOT = Path("__file__").resolve().parent.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

DB_PATH = ROOT / "database" / "retail.db"

# Consistent style for all charts
plt.rcParams.update({
    "figure.figsize": (10, 5),
    "axes.spines.top": False,
    "axes.spines.right": False,
    "axes.grid": True,
    "grid.alpha": 0.3,
})

print(f"Database path: {DB_PATH}")
print(f"Database exists: {DB_PATH.exists()}")

## 1. Load Data from SQLite

The pipeline created five tables. We'll use a helper function to query them.

In [ ]:
def query(sql: str) -> pd.DataFrame:
    """Run a SQL query against the retail database and return a DataFrame."""
    with sqlite3.connect(DB_PATH) as conn:
        return pd.read_sql_query(sql, conn)

# Load all tables
sales      = query("SELECT * FROM sales")
products   = query("SELECT * FROM products")
by_cat     = query("SELECT * FROM summary_by_category")
by_store   = query("SELECT * FROM summary_by_store")
by_date    = query("SELECT * FROM summary_by_date")

# Parse date column
sales["date"]    = pd.to_datetime(sales["date"])
by_date["date"]  = pd.to_datetime(by_date["date"])

print("Tables loaded:")
for name, df in [("sales", sales), ("products", products), ("by_cat", by_cat),
                 ("by_store", by_store), ("by_date", by_date)]:
    print(f"  {name:10s}  {df.shape[0]} rows × {df.shape[1]} cols")

## 2. Inspect the Sales Table

In [ ]:
sales.head()

In [ ]:
sales.dtypes

In [ ]:
# Check for any remaining nulls
sales.isnull().sum()

## 3. Summary Statistics

In [ ]:
sales[["quantity", "unit_price", "discount_pct",
       "gross_revenue", "net_revenue", "profit", "profit_margin_pct"]].describe().round(2)

In [ ]:
print(f"Total transactions : {len(sales)}")
print(f"Total units sold   : {sales['quantity'].sum()}")
print(f"Total gross revenue: ${sales['gross_revenue'].sum():,.2f}")
print(f"Total net revenue  : ${sales['net_revenue'].sum():,.2f}")
print(f"Total profit       : ${sales['profit'].sum():,.2f}")
print(f"Avg profit margin  : {sales['profit_margin_pct'].mean():.1f}%")

## 4. Revenue and Profit by Category

In [ ]:
by_cat

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Net revenue by category
axes[0].bar(by_cat["category"], by_cat["total_net_revenue"], color="steelblue")
axes[0].set_title("Net Revenue by Category")
axes[0].set_ylabel("Net Revenue ($)")
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

# Profit by category
axes[1].bar(by_cat["category"], by_cat["total_profit"], color="seagreen")
axes[1].set_title("Total Profit by Category")
axes[1].set_ylabel("Profit ($)")
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

## 5. Daily Sales Trend

In [ ]:
by_date.head()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Daily net revenue
axes[0].plot(by_date["date"], by_date["total_net_revenue"],
             marker="o", color="steelblue", linewidth=2)
axes[0].fill_between(by_date["date"], by_date["total_net_revenue"], alpha=0.15, color="steelblue")
axes[0].set_title("Daily Net Revenue")
axes[0].set_ylabel("Revenue ($)")
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

# Daily units sold
axes[1].bar(by_date["date"], by_date["total_units_sold"], color="coral", width=0.6)
axes[1].set_title("Daily Units Sold")
axes[1].set_ylabel("Units")
axes[1].set_xlabel("Date")

fig.autofmt_xdate(rotation=45)
plt.tight_layout()
plt.show()

## 6. Store Performance Comparison

In [ ]:
by_store

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

colors = ["steelblue", "seagreen", "coral"]

axes[0].bar(by_store["store_id"], by_store["total_net_revenue"], color=colors)
axes[0].set_title("Net Revenue per Store")
axes[0].set_ylabel("Revenue ($)")
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

axes[1].bar(by_store["store_id"], by_store["total_profit"], color=colors)
axes[1].set_title("Profit per Store")
axes[1].set_ylabel("Profit ($)")
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

axes[2].bar(by_store["store_id"], by_store["avg_profit_margin_pct"], color=colors)
axes[2].set_title("Avg Profit Margin per Store")
axes[2].set_ylabel("Margin (%)")

plt.tight_layout()
plt.show()

## 7. Top Products by Revenue

In [ ]:
top_products = (
    sales.groupby("name", as_index=False)
    .agg(
        transactions=("transaction_id", "count"),
        units_sold=("quantity", "sum"),
        net_revenue=("net_revenue", "sum"),
        profit=("profit", "sum"),
    )
    .round(2)
    .sort_values("net_revenue", ascending=False)
    .reset_index(drop=True)
)

top_products

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

bars = ax.barh(
    top_products["name"],
    top_products["net_revenue"],
    color="steelblue",
)

# Label each bar with the revenue value
for bar in bars:
    ax.text(
        bar.get_width() + 30,
        bar.get_y() + bar.get_height() / 2,
        f"${bar.get_width():,.0f}",
        va="center",
        fontsize=9,
    )

ax.invert_yaxis()  # highest revenue at the top
ax.set_title("Net Revenue by Product")
ax.set_xlabel("Net Revenue ($)")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x:,.0f}"))

plt.tight_layout()
plt.show()

## 8. Discount Impact Analysis

Do discounts correlate with higher quantities sold?

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

scatter = ax.scatter(
    sales["discount_pct"] * 100,
    sales["quantity"],
    c=sales["net_revenue"],
    cmap="YlOrRd",
    alpha=0.7,
    edgecolors="grey",
    linewidth=0.5,
    s=80,
)

plt.colorbar(scatter, ax=ax, label="Net Revenue ($)")
ax.set_title("Discount % vs Quantity Sold\n(colour = net revenue)")
ax.set_xlabel("Discount (%)")
ax.set_ylabel("Quantity Sold")

plt.tight_layout()
plt.show()

## 9. Customer Behaviour

How many transactions came from known vs unknown customers?

In [ ]:
customer_status = sales["customer_id"].apply(
    lambda x: "Unknown" if x == "UNKNOWN" else "Known"
).value_counts()

fig, ax = plt.subplots(figsize=(5, 5))
ax.pie(
    customer_status,
    labels=customer_status.index,
    autopct="%1.1f%%",
    colors=["steelblue", "lightcoral"],
    startangle=90,
)
ax.set_title("Transactions: Known vs Unknown Customer")
plt.tight_layout()
plt.show()

---
## Next Steps

Some questions worth exploring further:

- Which store has the highest profit margin — and why?
- Is there a day-of-week pattern in sales volume?
- Do high-discount transactions actually drive more revenue overall?
- Which products have the best margin, not just the highest revenue?

Try writing SQL queries against `database/retail.db` or pandas operations on the `sales` DataFrame to answer them!